# Modeling Demand

**Target variable**: `Proj_TRN_RoomsPickup`: How many transient rooms will be booked for each stay date, from this point (8/1/17) forward, at current prices?

In [1]:
import pandas as pd
import numpy as np

from agg import prep_demand_features
from demand_features import rf_cols

pd.options.display.max_rows = 160
pd.options.display.max_columns = 250
pd.options.display.max_colwidth = None

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import r2_score

from xgboost import XGBRegressor

DATE_FMT = "%Y-%m-%d"

from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV, HalvingGridSearchCV

print(__doc__)

Automatically created module for IPython interactive environment


In [2]:
h1_stats = pd.read_pickle("../data/h1_stats.pick")

In [3]:
len(rf_cols)

40

## Splitting Up Our Data for Train/Test

Our training set will contain all dates prior to as_of_date.

Our testing set will contain 31 stay dates starting on as_of_date. Our predictions will be used to provide price recommendations later on.

In [4]:
mask = (h1_stats["StayDate"] < '2017-08-01')
test_mask = (h1_stats['AsOfDate'] == '2017-08-01')
h1_train = h1_stats.loc[mask].copy()
h1_test = h1_stats.loc[test_mask].copy()

X1_train = h1_train[rf_cols].copy()
X1_test = h1_test[rf_cols].copy()
y1_train = h1_train['ACTUAL_TRN_RoomsPickup'].copy()
y1_test = h1_test['ACTUAL_TRN_RoomsPickup'].copy()

In [5]:
X1_train.shape

(11216, 40)

In [6]:
X1_train.head()

,week_of_year,RoomsOTB,RoomsOTB_STLY,TRN_RoomsOTB,TRN_RoomsOTB_STLY,WE,DaysUntilArrival,RemSupply,RemSupply_STLY,Mon,Sat,Sun,Thu,Tue,Wed,ACTUAL_RoomsPickup_STLY,ACTUAL_TRN_RoomsPickup_STLY,OTB_GapToLYA_RoomsSold,OTB_GapToLYA_TRN_RoomsSold,TM30_RoomsPickup,TM30_RoomsPickup_STLY,TM30_TRN_RoomsPickup,TM30_TRN_RoomsPickup_STLY,TM15_RoomsPickup,TM15_RoomsPickup_STLY,TM15_TRN_RoomsPickup,TM15_TRN_RoomsPickup_STLY,TM05_RoomsPickup,TM05_RoomsPickup_STLY,TM05_TRN_RoomsPickup,TM05_TRN_RoomsPickup_STLY,Pace_RoomsOTB,Pace_RemSupply,Pace_TRN_RoomsOTB,Pace_TM30_RoomsPickup,Pace_TM30_TRN_RoomsPickup,Pace_TM15_RoomsPickup,Pace_TM15_TRN_RoomsPickup,Pace_TM05_RoomsPickup,Pace_TM05_TRN_RoomsPickup
0,30.0,170.0,168.0,137.0,129.0,False,0.0,42.0,41.0,False,False,1,False,False,False,0.0,0.0,-2.0,-8.0,20.0,7.0,15.0,10.0,15.0,7.0,10.0,9.0,8.0,6.0,9.0,6.0,2.0,1.0,8.0,13.0,5.0,8.0,1.0,2.0,3.0
1,31.0,178.0,175.0,148.0,130.0,False,1.0,40.0,38.0,True,False,0,False,False,False,3.0,3.0,0.0,-15.0,3.0,3.0,3.0,6.0,3.0,7.0,3.0,9.0,7.0,2.0,7.0,2.0,3.0,2.0,18.0,0.0,-3.0,-4.0,-6.0,5.0,5.0
2,31.0,182.0,178.0,158.0,128.0,False,2.0,40.0,35.0,False,False,0,False,True,False,4.0,4.0,0.0,-26.0,2.0,1.0,2.0,2.0,2.0,6.0,2.0,6.0,0.0,2.0,0.0,2.0,4.0,5.0,30.0,1.0,0.0,-4.0,-4.0,-2.0,-2.0
3,31.0,174.0,175.0,152.0,130.0,False,3.0,51.0,40.0,False,False,0,False,False,True,7.0,7.0,8.0,-15.0,-5.0,1.0,-5.0,2.0,-2.0,1.0,-2.0,2.0,1.0,1.0,1.0,1.0,-1.0,11.0,22.0,-6.0,-7.0,-3.0,-4.0,0.0,0.0
4,31.0,179.0,176.0,149.0,133.0,False,4.0,50.0,41.0,False,False,0,True,False,False,4.0,4.0,1.0,-12.0,6.0,7.0,4.0,8.0,2.0,2.0,0.0,4.0,2.0,0.0,0.0,0.0,3.0,9.0,16.0,-1.0,-4.0,0.0,-4.0,2.0,0.0


## LINEAR REGRESSION

Results are not horrible, but it would not work this well in the real-world. Our target variable is not a linear combination of the rate & revenue features that we know have an impact on demand.

In [7]:
%%time
lm = LinearRegression()
lr_model = lm.fit(X1_train, y1_train)
scores = cross_val_score(lm, X1_train, y1_train, scoring='r2', cv=5)
scores.mean()

CPU times: user 3.54 s, sys: 4.12 s, total: 7.66 s
Wall time: 500 ms


0.7378753276596532

In [8]:
lr_model.score(X1_test, y1_test)

0.52736669230843

## RANDOM FOREST MODEL

I had high hopes for RF, and it came through. It works because of the amount and quality of the features I have engineered, despite the small training set. 

In [9]:
%%time
rfm = RandomForestRegressor(n_jobs=-1, random_state=20)
rf_model = rfm.fit(X1_train, y1_train)
scores = cross_val_score(rfm, X1_train, y1_train, scoring='r2', cv=5)
scores.mean()

CPU times: user 27.4 s, sys: 1.53 s, total: 28.9 s
Wall time: 10.8 s


0.7624015193342343

In [10]:
rf_model.score(X1_test, y1_test)

0.7380154149937472

with stly otb & cxl: 0.7309571151188048

without stly (pace only): .68

without cxl: 

## XGBOOST MODEL (GRADIENT BOOSTING TREES)

XGBoost came in close to RandomForest, though I fear how it will generalize with other hotels (less predictable ones, like city-hotel H2) due to the relatively small sample size of 1 year. 

In [11]:
%%time
xgbm = XGBRegressor(n_jobs=-1, random_state=20)
xgb_model = xgbm.fit(X1_train, y1_train)
scores = cross_val_score(xgbm, X1_train, y1_train, scoring='r2', cv=5)
scores.mean()

CPU times: user 1min 21s, sys: 115 ms, total: 1min 21s
Wall time: 5.27 s


0.7614100581142389

In [12]:
xgbm.score(X1_test, y1_test)

0.729108542819436

## MOVING FORWARD WITH RANDOM FOREST....

Below is how I tuned the hyperparameters. Suprisingly, even with over 100 CPU hours of grid search, I wasn't able to improve the $R^2$ score by more than 1% (both CV and test scores).


## Randomized Grid Search

Parameters of random grid search
```
random_grid = {
    "n_estimators": range(200, 2000, 100),
    "max_features": ["auto", "sqrt"],
    "max_depth": range(10, 110, 11),
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "bootstrap": [True, False]
}

rf = RandomForestRegressor()
rf_random = (RandomizedSearchCV(rf, random_grid, verbose=2, n_iter=50, random_state=42, n_jobs=-1))

rf_random.fit(X1_train, y1_train)
```

Results of random grid search:

```
{'n_estimators': 500,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 43,
 'bootstrap': True}
```

Score: 0.6519058137402494

## Brute Force Hyperparameter Tuning (GridSearchCV)

Best params thus far: 
Setup params:
```
GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [30, 56, 2],
                         'max_features': ['auto'],
                         'min_samples_split': [2, 3, 4, 8],
                         'n_estimators': range(300, 800, 40)},
             verbose=10)
```
Best resulting params:
```
{'bootstrap': True,
 'max_depth': 56,
 'max_features': 'auto',
 'min_samples_split': 3,
 'n_estimators': 300}
```

 $R^2$ CV score: `0.7785714200550233`


## Round 2


Param grid:
```
rf_grid = {
    "n_estimators": range(150, 500, 50),
    "max_features": ['auto'],
    "max_depth": range(32,56,2),
    "bootstrap": [True],
    "min_samples_split": [2, 3, 4]
}
```

And the **results**:
```
{'bootstrap': True,
 'max_depth': 48,
 'min_samples_split': 2,
 'n_estimators': 150}
```
$R^2$ CV score: `0.779336423856766`
 
### Round 3 (Worse than Round 2)

Param grid:
```
rf_grid = {
    "n_estimators": range(75, 225, 25),
    "max_depth": [47, 48, 49],
    "bootstrap": [True],
    "min_samples_split": [2],
}
```

And the **results**:

Best params:
```
{'bootstrap': True,
 'max_depth': 47,
 'min_samples_split': 2,
 'n_estimators': 125}
```
$R^2$ CV score: `0.7775378755829061`

In [19]:
rf_grid = {
    "n_estimators": [450, 465, 475, 485, 500],
    "max_depth": [28, 29, 30, 31, 32, None],
}
rfm = RandomForestRegressor()

rf_hgs = HalvingGridSearchCV(rfm, rf_grid, n_jobs=-1, verbose=10, cv=5, random_state=20)
rf_hgs.fit(X1_train, y1_train)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 415
max_resources_: 11216
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 30
n_resources: 415
Fitting 5 folds for each of 30 candidates, totalling 150 fits
----------
iter: 1
n_candidates: 10
n_resources: 1245
Fitting 5 folds for each of 10 candidates, totalling 50 fits
----------
iter: 2
n_candidates: 4
n_resources: 3735
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 3
n_candidates: 2
n_resources: 11205
Fitting 5 folds for each of 2 candidates, totalling 10 fits


HalvingGridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
                    param_grid={'max_depth': [28, 29, 30, 31, 32, None],
                                'n_estimators': [450, 465, 475, 485, 500]},
                    random_state=20,
                    refit=<function _refit_callable at 0x7f8e73195830>,
                    verbose=10)

In [15]:
rf_hgs.best_params_

{'max_depth': 28, 'n_estimators': 475}

In [16]:
rf_hgs.best_score_

0.7643058943002272

In [17]:
rf_hgs.score(X1_test, y1_test)

0.739915104194734

In [ ]:
## Final param tuning with brute force grid search

In [21]:
rf_grid = {
    "n_estimators": [100, 550],
    "max_depth": [18, 20, 22, 24, None],
}
rfm = RandomForestRegressor()

rfg = GridSearchCV(rfm, rf_grid, n_jobs=-1, verbose=10, cv=5)
rfg.fit(X1_train, y1_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [18, 20, 22, 24, None],
                         'n_estimators': [100, 550]},
             verbose=10)

In [22]:
print(f"""Grid Search Results:

Optimal parameters: {rfg.best_params_}
Best CV score:      {rfg.best_score_}""")

Grid Search Results:

Optimal parameters: {'max_depth': None, 'n_estimators': 550}
Best CV score:      0.7655277443765263


In [23]:
rfg.score(X1_test, y1_test)

0.7398501647942479

## Final Model

In [ ]:
rf = RandomForestRegressor(n_estimators=550, n_jobs=-1, random_state=20)

rf.fit(X1_train, y1_train)

In [ ]:
rf.score(X1_test, y1_test)

## Now that we have our model, let's get it in the simulation so we can evaluate our results.

Head over to `demand_model_evaluation.ipynb` for more.